In [35]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import  Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation,Dense, Flatten, MaxPooling2D,Conv2D, Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:
from google.colab import drive

In [25]:
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [26]:
dataTrain_dir='MyDrive/MyDrive/cats_and_dogs_filtered/cats_and_dogs_filtered/train'
dataVal_dir='MyDrive/MyDrive/cats_and_dogs_filtered/cats_and_dogs_filtered/validation'

In [27]:
#Preprocessing
train_aug=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

In [28]:
train_generator= train_aug.flow_from_directory(dataTrain_dir,
    target_size=(128, 128),
    class_mode='binary',
    batch_size=128,
)

Found 2000 images belonging to 2 classes.


In [29]:
#Rescaling images
val_aug=ImageDataGenerator(rescale=1./255)


In [30]:
val_gnerator=val_aug.flow_from_directory(dataVal_dir,
    target_size=(128, 128),
    class_mode='binary',
    batch_size=128
)

Found 1000 images belonging to 2 classes.


In [31]:
#VGG16 Pretrained Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
conv_base= VGG16(input_shape=(128,128,3),include_top=False,weights='imagenet')

In [32]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [33]:
#Freeze output layers in conv_base
for layers in conv_base.layers:
    layers.trainable=False

In [36]:
#Create output layers for model
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [37]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               2097408   
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [38]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [40]:
history=model.fit(train_generator,
                  steps_per_epoch=8,
                  epochs=5,
                  verbose=1,
                  validation_data=val_gnerator)

Epoch 1/5
8/8 [==============================] - 515s 67s/step - loss: 3.6770 - accuracy: 0.5236 - val_loss: 1.1906 - val_accuracy: 0.5010
Epoch 2/5
8/8 [==============================] - 383s 50s/step - loss: 1.0083 - accuracy: 0.5078 - val_loss: 0.6317 - val_accuracy: 0.6450
Epoch 3/5
8/8 [==============================] - 344s 46s/step - loss: 0.6049 - accuracy: 0.6680 - val_loss: 0.6356 - val_accuracy: 0.6540
Epoch 4/5
8/8 [==============================] - 379s 51s/step - loss: 0.6067 - accuracy: 0.6738 - val_loss: 0.4177 - val_accuracy: 0.7960
Epoch 5/5
8/8 [==============================] - 349s 46s/step - loss: 0.7736 - accuracy: 0.6055 - val_loss: 0.6293 - val_accuracy: 0.6880
